# Modeling (otw)
- masih belum fine-tuning pretrained model

## Import Library & Datasets

In [8]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# load content_tokens
content_tokens = []
with open('content_tokens.txt', 'r') as f:
    for line in f:
        content_tokens.append(line.strip())

content_tokens[:15]

['[CLS]',
 '“',
 'infeksi',
 'co',
 '##vid',
 '-',
 '19',
 'adalah',
 'salah',
 'satu',
 'penyakit',
 'yang',
 'perlu',
 'diwaspadai',
 '.']

## Feature Extraction 
(small preprocessing) sebelum dataset digunakan

In [ ]:
# Load the pre-trained model and tokenizer
model_name = 'indobenchmark/indobert-base-p2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# Get the embeddings of the content
content_embeddings = []
batch_size = 8

for i in range(0, len(content_tokens), batch_size):
    batch = content_tokens[i:i+batch_size]
    encoded_inputs = tokenizer.batch_encode_plus(batch, padding=True, truncation=True, max_length=max_sequence_length, return_tensors='pt')
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state[:, 0, :]
        content_embeddings.append(embeddings)

content_embeddings = torch.cat(content_embeddings)

## Modeling

In [ ]:
# Function to perform information retrieval
def retrieve_documents(query, k=3):
    query_tokens = tokenizer.tokenize(query)
    query_tokens = ['[CLS]'] + query_tokens + ['[SEP]']
    encoded_query = tokenizer.encode_plus(query_tokens, padding=True, truncation=True, max_length=max_sequence_length, return_tensors='pt')
    input_ids = encoded_query['input_ids']
    attention_mask = encoded_query['attention_mask']
    with torch.no_grad():
        query_outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        query_embedding = query_outputs.last_hidden_state[:, 0, :]
    similarities = cosine_similarity(query_embedding, content_embeddings)[0]
    top_indices = similarities.argsort()[::-1][:k]
    top_documents = df.loc[top_indices]
    return top_documents

# Example usage
query = 'Your query here...'
top_docs = retrieve_documents(query, k=3)
print(top_docs[['title', 'content']])

In [3]:
# read valid.json

import json
import os
import sys
import time

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

validation = pd.read_json('valid.json', lines=True)

0    {'id': 3721, 'text': 'Liputan6 . com , Jakarta...
Name: 0, dtype: object